In [1]:
import pickle
import gensim
import pandas as pd
import numpy as np
import re
import torch
from tqdm import tqdm

In [2]:
tweets = pd.read_pickle('../data/interim/hydrated/200316.pkl')
tweets

,tweet_id,created_at,text
0,1239340650824335362,2020-03-16 00:00:11,Large-scale testing for Covid-19 and wide avai...
1,1239340642888749057,2020-03-16 00:00:09,Stories on covid_19 😱😱😱
2,1239340642695884810,2020-03-16 00:00:09,"Smart. More of this, please. https://t.co/c7Tk..."
3,1239340637847093249,2020-03-16 00:00:08,Not going to wear a bra for a month. 🙅🏻‍♀️✨\n\...
4,1239340613398495232,2020-03-16 00:00:02,A COVID-19 coronavirus update from concerned p...
...,...,...,...
376227,1239702984109350912,2020-03-16 23:59:58,The Wall Street Journal keeps calling the coro...
376228,1239702984243720194,2020-03-16 23:59:58,"Time to put on like fifteen pounds, or possibl..."
376229,1239702988404535297,2020-03-16 23:59:59,Idris Elba Has Tested Positive for Coronavirus...
376230,1239702984273145856,2020-03-16 23:59:58,This is how you be a millionaire/billionaire. ...


In [3]:
file = open('../data/processed/Embedding.pickle', 'rb')
word2vec = pickle.load(file)

In [4]:
tweets.text[0]

'Large-scale testing for Covid-19 and wide availability of testing on demand would seem a no-brainer in early detection and control strategies, as in South Korea. So it seems bizarre it has now been restricted to only suspected serious cases referred to hospitals in UK. https://t.co/gtsxzzS9qA'

In [5]:
rsum = np.zeros(300)
 
for word, embedding in word2vec.vocab.items():
    rsum += word2vec[word]
 
average = rsum/len(word2vec.vocab)

KeyboardInterrupt: 

In [22]:
word2vec.similar_by_vector(average, topn=10)

[('Al-Department', 0.9518683552742004),
 ('Interrogée', 0.9515928626060486),
 ('www.optimization.co.jp', 0.9513416290283203),
 ('2016Table', 0.9502310156822205),
 ('Lamorisse.', 0.9487457871437073),
 ('Tehut', 0.9480428695678711),
 ('www.fxcn.co.jp', 0.9471553564071655),
 ('www.sky-value.com', 0.946313202381134),
 ('www.wei-xiu.org.cn', 0.945926308631897),
 ('TURNBERRY', 0.9456648230552673)]

# Regex

In [46]:
character_set = {
    "characters": "abcdefghijklmnopqrstuvwxyz0123456789 ",
    "end_string": "<EOS>",
    "space": " ",
}

regex_html_tags = {
    "&amp;": "and",
    "&lt;": "<",
    "&gt;": ">",
    "&quot;": '"',
    "&apos;": "'",
}

regex_prefix_user_name = re.compile("^(?:@\S+\s)+")
regex_inner_user_name = re.compile("@\S+")
regex_links = re.compile("http\S+")
regex_whitespace = re.compile("[\s|-|-|_]+")
regex_unknown = re.compile(f"[^{ character_set['characters'] }]+")


In [47]:
for pattern_string, char in regex_html_tags.items():
    tweets["text_processed"] = tweets["text"].str.replace(pattern_string, char)

In [55]:
tweets["text_processed"] = tweets["text_processed"] = (
    tweets["text_processed"]
    .str.lower()
    .str.replace(regex_whitespace, character_set["space"])
    .str.strip()
    .str.replace(regex_prefix_user_name, "")
    .str.strip()
    .str.replace(regex_inner_user_name, "")
    .str.strip()
    .str.replace(regex_links, "")
    .str.strip()
    .str.replace(regex_unknown, "")
    .str.strip()
)

In [121]:
# Drop empty tweets
tweets = tweets[tweets.text_processed != '']
tweets.to_pickle('../data/interim/hydrated/200316_text_processed.pkl')

In [128]:
tweets_embedding = []
for tweet in tqdm(tweets["text_processed"], total=len(tweets)):

    embedding_tweet = torch.from_numpy(
        np.vstack(
            [word2vec[word] if word in word2vec.vocab.keys() else average for word in tweet.split()],
        ),
    )
    embedding_tweet = embedding_tweet.type(torch.float16)
    tweets_embedding.append(embedding_tweet)

100%|██████████| 376230/376230 [01:19<00:00, 4735.92it/s]


In [129]:
with open('../data/processed/200316_embedding.pkl', 'wb') as f:
    pickle.dump(tweets_embedding, f)

OSError: [Errno 22] Invalid argument

In [1]:
from src.data.hydrate import _pickle_object, _unpickle_object

In [7]:
tmp = torch.load('../data/processed/200316_embedding.pkl')

In [8]:
from src.data.words import TwitterDataWords

In [9]:
ds = TwitterDataWords( tmp )

In [11]:

ds[[2,3 ,4 ,5]]

RuntimeError: The expanded size of the tensor (5) must match the existing size (11) at non-singleton dimension 1.  Target sizes: [300, 5].  Tensor sizes: [300, 11]

In [ ]:
tmp[0]